### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using aql.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Runing aql in iJava
We can run aql commands in a file. No output will be displayed. 
In _trucate_test.aql_, we have a single AQL command: TRUNCATE test



In [67]:
%sh aql -c "truncate test" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.0.0</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

import com.aerospike.client.Operation;

import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.ExpOperation;
import com.aerospike.client.exp.ExpWriteFlags;
import com.aerospike.client.exp.Expression;
import com.aerospike.client.exp.ExpReadFlags;

import java.util.Random;



In [68]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);

//Initialize our single aggregation record, center of circle 

Key key1 = new Key("test", "points", 1); 
Bin xcoord = new Bin("x", 0.0);
Bin ycoord = new Bin("y", 0.0);
Bin inCount = new Bin("inCount", 1.0);
Bin totalCount = new Bin ("count", 1.0);
WritePolicy wPolicy = new WritePolicy();
client.put(wPolicy, key1, xcoord, ycoord, inCount, totalCount );
//Check
System.out.println(client.get(null, key1));



(gen:1),(exp:448491015),(bins:(x:0.0),(y:0.0),(inCount:1.0),(count:1.0))


In [45]:
//Insert a record using a class based construct
class Point {
    
  public void insertPoint (Key key1, double xval, double yval, boolean bCheck) {
    Bin xcoord = new Bin("x", xval);
    Bin ycoord = new Bin("y", yval);
    WritePolicy wPolicy1 = new WritePolicy();
      
    Expression inCircleExp = Exp.build(
       Exp.cond(
        Exp.le(
          Exp.add(
            Exp.mul(Exp.floatBin("x"), Exp.floatBin("x")), 
            Exp.mul(Exp.floatBin("y"), Exp.floatBin("y"))
          ),
          Exp.val(1.0)
        ),
       Exp.add(Exp.floatBin("inCount"), Exp.val(1.0)),
       Exp.floatBin("inCount")
       ));
        
    Expression totalCountExp = Exp.build(Exp.add(Exp.floatBin("count"), Exp.val(1.0)));
 
    Record record = client.operate( wPolicy1, key1,   
          Operation.put(xcoord),
          Operation.put(ycoord),                   
          ExpOperation.write("inCount", inCircleExp, ExpWriteFlags.DEFAULT),
          ExpOperation.write("count", totalCountExp, ExpWriteFlags.DEFAULT)
                                  );                 
    if(bCheck){
      System.out.println(client.get(null, key1));
    }   
  }
    
  public void getPi(Key key1){
      WritePolicy wPolicy = new WritePolicy();
      Expression piExp = Exp.build(
          Exp.mul(Exp.val(4.0), 
                  Exp.div(Exp.floatBin("inCount"),Exp.floatBin("count"))
                 ));
      
      Record record = client.operate(wPolicy, key1, ExpOperation.read("pi", piExp, ExpReadFlags.DEFAULT));
      System.out.println("Pi : " + record.getValue("pi"));
  }
  
}


In [46]:
//Check
Point p = new Point();
p.insertPoint(key1, 0.71, 0.71, true);  //Out of circle, in rectangle

(gen:2),(exp:448490510),(bins:(x:0.71),(y:0.71),(inCount:1.0),(count:2.0))


In [69]:
//Insert random points with x,y between 0.0 and 1.0

Random rnd1 = new Random(0);  

for(int i=1;i<10000;i++){
  double xval = rnd1.nextDouble();
  double yval = rnd1.nextDouble();
  p.insertPoint(key1, xval, yval, false);  
}

In [70]:
p.getPi(key1);

Pi : 3.1424
